In [1]:
import pandas as pd

In [2]:
cs=pd.read_csv('cleaned_data.csv')

In [3]:
df=pd.DataFrame(cs)

In [4]:
df.head(5)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there is a family where a little boy...,negative
4,petter matteis love in the time of money is a ...,positive


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
import data_preprocessing as ps
import re

C:\Users\sai mahanth\anaconda3\envs\project\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
# DATA CLEANING
def data_cleaning(x):
    x=str(x).lower().replace('\\','').replace('_',' ').replace('<br />',' ')
    x=ps.cont_exp(x) #expansion of sentences
    x=ps.remove_urls(x)
    x=ps.remove_html_tags(x)
    x=ps.remove_accented_chars(x)
    x=ps.remove_special_chars(x)
    x=re.sub("(.)\\1{2,}","\\1",x) #removal of multi charcters in data
    return x

df['review']=df['review'].apply(lambda x: data_cleaning(x))
df.head(4)
df.to_csv('cleaned_data.csv')

In [8]:
def pto1(y):
    y=str(y).replace('positive','1').replace('negative','0')
    y=int(y)
    return y
df['sentiment']=df['sentiment'].apply(lambda y: pto1(y))

In [9]:
df.head(5)

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there is a family where a little boy...,0
4,petter matteis love in the time of money is a ...,1


In [10]:
df_x=df['review']
df_y=df['sentiment']
x_train,x_test,y_train,y_test=train_test_split(df_x,df_y,test_size=0.2,random_state=0)

In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence

In [12]:
stopwords=set(stopwords.words('english'))

In [13]:
res=[]
my_vocab_size=100000
def encod(z):
    enc=[]
    z=word_tokenize(z)
    new_words=[]
    for w in z:
        if w not in stopwords:
            new_words.append(w)
    for i in new_words:
        enc.append(one_hot(i,my_vocab_size)[0])
    res.append(enc)

In [14]:
x_train.apply(lambda z : encod(z))
x_tr=res
res=[]
x_test.apply(lambda z : encod(z))
x_te=res

In [15]:
x_tr=np.array(x_tr)
x_te=np.array(x_te)

<ipython-input-15-32be2675650a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_tr=np.array(x_tr)
<ipython-input-15-32be2675650a>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_te=np.array(x_te)


In [16]:
max_review_length = 500
x_tr = sequence.pad_sequences(x_tr, maxlen=max_review_length)
x_te = sequence.pad_sequences(x_te, maxlen=max_review_length)

In [17]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding

In [19]:
model = Sequential()
model.add(Embedding(100000, 64,input_length=500))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(x_tr, y_train, validation_data=(x_te, y_test), epochs=2, batch_size=64)

Epoch 1/2
625/625 [==============================] - 436s 697ms/step - loss: 0.3621 - accuracy: 0.8408 - val_loss: 0.2849 - val_accuracy: 0.8876
Epoch 2/2
625/625 [==============================] - 445s 711ms/step - loss: 0.1629 - accuracy: 0.9417 - val_loss: 0.2997 - val_accuracy: 0.8809


In [22]:
scores = model.evaluate(x_te, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.09%


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 64)           6400000   
_________________________________________________________________
dropout (Dropout)            (None, 500, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               66000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 6,466,101
Trainable params: 6,466,101
Non-trainable params: 0
_________________________________________________________________


In [24]:
text1='i liked the movie , the movie was awesome and i loved it'

In [25]:
cleaned1=data_cleaning(text1)

In [26]:
res=[]
encod(cleaned1)
x_t=res
x_t=np.array(x_t)

In [27]:
x_t=sequence.pad_sequences(x_t, maxlen=max_review_length)

In [28]:
model.predict_classes(x_t)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1]])

In [29]:
model.predict(x_t)

array([[0.98024046]], dtype=float32)

In [30]:
text2='this was disgusting and worse that it was so dumb'

In [31]:
cleaned2=data_cleaning(text2)

In [32]:
res=[]
encod(cleaned2)
y_t=res
y_t=np.array(y_t)

In [33]:
y_t=sequence.pad_sequences(y_t, maxlen=max_review_length)

In [34]:
model.predict_classes(y_t)

array([[0]])

In [35]:
model.predict(y_t)

array([[0.08350688]], dtype=float32)

In [36]:
from keras.models import load_model

In [37]:
model.save('final1.h5')

In [38]:
#new_model=load_model('final.h5')